# **Conectar con drive para acceder a los datos**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Imports**

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import requests
import operator
from collections import Counter
from collections import defaultdict
import math
from traitlets.traitlets import default
import pickle
from datetime import timedelta
import seaborn as sns
from tabulate import tabulate
import statistics

In [3]:
path = 'yourpath.pickle'
with open(path, 'rb') as f:
  all_dao_info = pickle.load(f)

#print(all_dao_info)

In [4]:
#Periodo de tiempo a estudiar
ini = min(all_dao_info["df_votes"]["created"])
fin = datetime(2023, 1, 1, 0, 0, 0, 0)
allMonths = pd.date_range(start=ini, end=fin, freq=pd.DateOffset(months=1)).strftime("%Y-%m")
allMonthsNew = pd.date_range(start=ini, end=fin, freq=pd.DateOffset(months=1)).strftime("%Y-%b")

In [5]:
#eje date

DateAxis = []
for k in allMonthsNew:
  sp = k.split("-")
  if sp[1] != "Jan":
    DateAxis.append(sp[1])
  else:
    DateAxis.append(sp[1]+"\n"+sp[0])

# print(ini)
# print(DateAxis)

In [6]:
#ordenar los id de propuesta por fecha
#asi podemos estar 100% seguros de que recorremos props en orden cronologico
tupla = zip(all_dao_info['df_proposals']['id'], all_dao_info['df_proposals']['date'])
IDsPropsOrdenados = sorted(tupla, key=lambda date : date[1])

In [7]:
plt.rcParams["figure.figsize"]=(12,7)

# **Métricas sin gráficas**

In [ ]:
print(f"Total number of proposals: {all_dao_info['df_proposals']['id'].count()}")
print(f"Total number of authors: {all_dao_info['num_authors']}")
print(f"Total number of voters: {all_dao_info['num_voters']}")

print(f"Participation: {len(set(all_dao_info['voters_list']).union(set(all_dao_info['authors_list'])))}")



In [ ]:
#votos
print(f"Total number of votes: {all_dao_info['df_votes']['id'].count()}")

maxi = 0
mini = 2147483647
myList = []

for prop_id in all_dao_info['df_proposals']['id']:
  n = list(all_dao_info['df_proposals'][all_dao_info['df_proposals']['id'] == prop_id]['votes'])[0]
  if n > maxi:
    maxi = n
  if n < mini and n>0:
    mini = n
  myList.append(n)

print(f"Max number of votes in a proposal: {maxi}")
print(f"Min number of votes in a proposal: {mini}")
print(f"Median number of votes: {statistics.median(myList)}")
print(f"Mean number of votes: {statistics.mean(myList)}")

In [ ]:
#votos por votante

votos_gr = all_dao_info['df_votes'].groupby('voter')['id'].count()

print(f"Max number of votes for a voter: {max(votos_gr)}")
print(f"Min number of votes for a voter: {min(votos_gr)}")
print(f"Median number of votes: {statistics.median(votos_gr)}")
print(f"Mean number of votes: {statistics.mean(votos_gr)}")

In [ ]:
#suma de las medianas de vp de los votantes
vp = []

for voter in all_dao_info['voters_list']:
  vp.append(statistics.median(all_dao_info['df_votes'][all_dao_info['df_votes']['voter'] == voter]['vp']))

total_median_vp = sum(vp)

print(f"Suma de las medianas de vp de los votantes: {total_median_vp}")

In [ ]:
#VP propuesta
print(f"Total number of vp: {all_dao_info['df_proposals']['scores_total'].sum()}")

maxi = 0
mini = 2147483647
myList = []

for prop_id in all_dao_info['df_proposals']['id']:
  n = list(all_dao_info['df_proposals'][all_dao_info['df_proposals']['id'] == prop_id]['scores_total'])[0]
  if n > maxi:
    maxi = n
  if n < mini:
    mini = n
  myList.append(n)

print(f"Max vp in a proposal: {maxi}, {maxi/total_median_vp*100}")
print(f"Min vp in a proposal: {mini}, {mini/total_median_vp*100}")
print(f"Median vp: {statistics.median(myList)}, {statistics.median(myList) /total_median_vp*100}")
print(f"Mean vp: {statistics.mean(myList)}, {statistics.mean(myList)  /total_median_vp*100}")

In [ ]:
#voting power por votante
vp_gr = all_dao_info['df_votes'].groupby('voter')['vp'].median()

print(f"Max vp for a voter (median): {max(vp_gr)}, {max(vp_gr)  /total_median_vp*100}")
print(f"Min vp for a voter (median): {min(vp_gr)}, {min(vp_gr) /total_median_vp*100}")
print(f"Median vp for a voter (median): {statistics.median(vp_gr)}, {statistics.median(vp_gr) /total_median_vp*100}")
print(f"Mean vp for a voter (median): {statistics.mean(vp_gr)}, {statistics.mean(vp_gr) /total_median_vp*100}")

In [ ]:
#resultado propuestas
secundada = 0
nosecundada = 0
novotos = 0
unanimidadConAutor = 0
unanimidadSinAutor = 0

for prop_id in all_dao_info['df_proposals']['id']:
  autor = list(all_dao_info['df_proposals'][all_dao_info['df_proposals']['id'] == prop_id]['author'])[0]

  autor_aux = all_dao_info['df_votes']['voter'] == autor
  prop_aux = np.array(list(map(lambda x: x['id'], all_dao_info['df_votes']['proposal']))) == prop_id

  num_votos = list(all_dao_info['df_proposals'][all_dao_info['df_proposals']['id'] == prop_id]['votes'])[0]
  filter = all_dao_info['df_votes'][ np.logical_and(prop_aux , autor_aux)]['created']

  #gris: nadie vota
  if num_votos == 0:
    novotos +=1
  elif len(filter) > 0: #el autor vota
    opt_autor = all_dao_info['df_votes'][ np.logical_and(prop_aux , autor_aux)]['choice'].to_list()[0]
    win = all_dao_info['df_proposals'][all_dao_info['df_proposals']['id'] == prop_id]['scores'].to_list()[0]
    maxi = max(win)
    opt_winner = [x for x in range(len(win)) if win[x] == maxi][0]
    n = all_dao_info['results_per_proposals'][prop_id][opt_autor]

    #verde -> autor vota la opcion que gana
    if opt_winner == opt_autor-1 :
      secundada += 1
      if n == num_votos:
        unanimidadConAutor +=1
    #rojo
    else:
      nosecundada +=1
  else:
    #el autor no vota pero la gente si, mirar unanimidad
    for key, val in all_dao_info['results_per_proposals'][prop_id].items():
      if val == num_votos:
        unanimidadSinAutor +=1

print(f"Propuestas secundadas: {secundada}")
print(f"Propuestas no secundadas: {nosecundada}")
print(f"Propuestas sin votos: {novotos}")
print(f"Propuestas con unanimidad: {unanimidadSinAutor + unanimidadConAutor}")

In [ ]:
#vp opcion ganadora

props = all_dao_info["df_proposals"]

vp = []

for i in range(len(props)):
    maxi = 0
    total = 0
    for j in range(len(props['choices'][i])):
      if props['scores'][i][j] > maxi:
        maxi = props['scores'][i][j]
      total += props['scores'][i][j]

    if total == 0:
      continue
    porcentaje = maxi * 100 / total
    vp.append(porcentaje)

print(f"VP medio opcion ganadora: {statistics.mean(vp)}")

In [ ]:
#votos opcion ganadora
votos = []
for prop_id in all_dao_info['df_proposals']['id']:
  if num_votos != 0:
    maxi = 0
    total = 0
    for key, val in all_dao_info['results_per_proposals'][prop_id].items():
      if val > maxi:
        maxi = val
      total += val
    if total == 0:
      continue

    porcentaje = maxi * 100 / total
    votos.append(porcentaje)


print(f"Votos medio opcion ganadora: {statistics.mean(votos)}")

In [ ]:
#votantes que acumulan >50% VP (nº y %)
totalVP = 0
vps = []
for voter in all_dao_info['voters_list']:
  sum_vp = all_dao_info["df_votes"][all_dao_info["df_votes"]['voter']==voter]['vp'].median()
  totalVP += sum_vp
  vps.append(sum_vp)

ordenado = sorted(vps, reverse = True)

sumi = 0
for i in range(len(ordenado)):
  sumi += ordenado[i]
  if sumi > totalVP/2:
    print("Votantes que acumulan > 50")
    res = i+1
    print(res)
    break

print("Porcentaje")
print(res/len(ordenado)*100)

# **Gráficas**


In [ ]:
# Distribución del poder de los votantes.
myDict={}
test=[]

for voter in all_dao_info['voters_list']:
  mymax = max(all_dao_info["df_votes"][all_dao_info["df_votes"]['voter']==voter]['vp'])
  mymin = min(all_dao_info["df_votes"][all_dao_info["df_votes"]['voter']==voter]['vp'])
  mymedian = statistics.median(all_dao_info["df_votes"][all_dao_info["df_votes"]['voter']==voter]['vp'])
  # if len(all_dao_info["df_votes"][all_dao_info["df_votes"]['voter']==voter]['vp']) > 1:
  #   if mymax > mymin:
  myDict[voter] = (mymax+1, mymin+1, mymedian+1)
  test.append(mymedian)
print(test)
print(statistics.median(test))

ordenado = sorted(myDict.items(), key = lambda x:x[1][2])
plt.rcParams["figure.figsize"]=(12,7)
for i in range(len(list(map(lambda p: p[0], ordenado)))):
   plt.vlines(i+1, list(map(lambda p: p[1][1]/(10**6), ordenado))[i], list(map(lambda p: p[1][0]/(10**6), ordenado))[i], color='black', zorder=0)

plt.scatter(range(1, len(list(map(lambda p: p[0], ordenado)))+1), list(map(lambda p: p[1][2]/(10**6), ordenado)), zorder=5)
plt.ylim(0, 1400000/(10**6))
plt.xlim(-.95, 65)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Votantes", fontsize=20)
plt.ylabel("Poder de voto (millones)", fontsize=20)

plt.savefig("distribucionVP_Pillar.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
# Curva de Lorenz.
def lorenz_curve(X):
  X_lorenz = X.cumsum() / X.sum()
  X_lorenz = np.insert(X_lorenz, 0, 0)
  X_lorenz[0], X_lorenz[-1]

  fig, ax = plt.subplots(figsize=[6,6])
  ## scatter plot of Lorenz curve
  ax.scatter(np.arange(X_lorenz.size)/(X_lorenz.size-1), X_lorenz,
           s=80, facecolors='none', edgecolors='darkorange')
  ## line plot of equality
  ax.plot([0,1], [0,1], color='k')
  plt.axhline(y=0.5, color='grey', lw=0.5)

  plt.ylabel("Poder de voto acumulado" , fontsize=20)
  plt.xlabel("Votantes acumulados", fontsize=20)

  plt.xticks(fontsize=15)
  plt.yticks(fontsize=15)

  plt.savefig("lorenz.pdf", format="pdf", bbox_inches="tight")

  plt.show()



X = dict(all_dao_info["df_votes"].groupby(["voter"])["vp"].median())
X = np.array(sorted(list(X.values())))


lorenz_curve(X)


In [ ]:
#scatter plot: Relación entre el número de votos que emite un votante y su poder de voto
VP_gruped_per_voter = all_dao_info["df_votes"].groupby(["voter"])["vp"]
VP_median_per_voter = VP_gruped_per_voter.median()

votes_per_voter = all_dao_info["df_votes"].groupby(["voter"])["id"].count()


plt.style.use('bmh')
plt.scatter(VP_median_per_voter, votes_per_voter, alpha=0.25, s=75)
plt.xlabel('Poder de voto mediano', fontsize=20)
plt.ylabel('Número de votos emitidos', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.savefig("fig.pdf", format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
#GINI para vp

def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))



VP_gruped_per_voter = all_dao_info["df_votes"].groupby(["voter"])["vp"]
data = VP_gruped_per_voter.median()
print("The Gini coefficient is (groupped by voter): ", gini(data))


In [ ]:
#datos gini para tabla
data = []
i=0
n=0
for prop_id, date in IDsPropsOrdenados:
  #si la propuesta ha recibido voting power
  if list(all_dao_info['df_proposals'][all_dao_info['df_proposals']['id'] == prop_id]['scores_total'])[0] > 0:
    #cojo los votos de la propuesta
    prop_aux = np.array(list(map(lambda x: x['id'], all_dao_info['df_votes']['proposal']))) == prop_id
    #saco vp de los votos
    total_vp = all_dao_info['df_votes'][ prop_aux ]['vp']
    if len(total_vp) > 1:
      val = gini(np.array(total_vp))
      data.append(val)



print(f'Max = {max(data)}')
print(f'min = {min(data)}')
print(f'mean = {statistics.mean(data)}')
print(f'median = {statistics.median(data)}')

In [ ]:
#gini por propuesta
plt.rcParams["figure.figsize"]=(16,10)
data = {}
cadena = "P"
i = 1

for prop_id, date in IDsPropsOrdenados:
  if list(all_dao_info['df_proposals'][all_dao_info['df_proposals']['id'] == prop_id]['scores_total'])[0] != 0:
    prop_aux = np.array(list(map(lambda x: x['id'], all_dao_info['df_votes']['proposal']))) == prop_id
    total_vp = all_dao_info['df_votes'][ prop_aux ]['vp']
    data[cadena + str(i)] = gini(np.array(total_vp))
  i+=1

plt.bar(list(data.keys()), list(data.values()))
plt.ylim(0,1)
plt.xticks(rotation=45)
plt.xlabel('Propuesta', fontsize=25)
plt.ylabel('Coeficiente de Gini', fontsize=25)
plt.xticks(fontsize=12)
plt.yticks(fontsize=20)

plt.savefig("gini_pillar.pdf", format="pdf", bbox_inches="tight")



In [ ]:
#Porcentaje de votos y poder de voto que recibe la opción ganadora de las propuestas.
props = all_dao_info["df_proposals"]

vp = {}
nvotos = {}
x = 1
for elem, nousar in IDsPropsOrdenados:
    maxi = 0
    total = 0
    for j in range(len(list(props[all_dao_info['df_proposals']['id'] ==  elem]['choices'])[0])):
      if list(props[all_dao_info['df_proposals']['id'] ==  elem]['scores'])[0][j] > maxi:
        maxi = list(props[all_dao_info['df_proposals']['id'] ==  elem]['scores'])[0][j]
      total += list(props[all_dao_info['df_proposals']['id'] ==  elem]['scores'])[0][j]

    if total == 0:
      x+=1
      continue
    #% opt ganadora
    porcentaje = maxi * 100 / total

    vp[ "P"+str(x)] = porcentaje
    x+=1


#nvotos
#no mismo orden
aux = {}
for prop, val in all_dao_info['results_per_proposals'].items():
    if sum(val.values()) == 0:
      continue
    perc = max(val.values())*100/sum(val.values())
    aux[prop] = perc

#for para ordenar
y = 1
for prop, nousar in IDsPropsOrdenados:
  if prop in aux:
    nvotos["P"+str(y)] = aux[prop]
    #print(prop)
  y+=1


#plot
plt.style.use('bmh')


x_axis = np.arange(len(list(map(lambda p: p[0], nvotos))))
fig = plt.figure()
ax = plt.gca()


ax.bar( x_axis-0.2, list(nvotos.values()), width=0.4, align='center', color='tab:orange', label='Votos')
plt.xticks(x_axis, list(nvotos.keys()), rotation=45)

plt.xlabel('Propuestas', fontsize=20)
plt.ylabel('Porcentaje', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
ax.tick_params(axis='y')

ax.bar(  x_axis+0.2, list(vp.values()), width=0.4, align='center', color='tab:blue', label='Poder de voto')
plt.legend(fontsize=20, loc=4)


plt.savefig("agree_bar_Pillar.pdf", format="pdf", bbox_inches="tight")
plt.show()



In [ ]:
# Scatter plot Relación entre los porcentajes de votos y poder de voto que recibe la opción ganadora de las propuesta

#Para la misma propuesta necesito su %de votos y su % de vp

#Lista de id-propuesta, % de votos opcion ganadora
myDictVotos = {}
for prop, val in all_dao_info['results_per_proposals'].items():
 # if len(val) == 2:
    if sum(val.values()) != 0:
      percVotos = max(val.values())*100/sum(val.values())
      myDictVotos[prop]=percVotos

#Lista de id-propuesta, % de vp opcion ganadora
#esta en scores
myDictVP = {}
for i in range(len(all_dao_info["df_proposals"])):
  #if len(all_dao_info["df_proposals"]["scores"][i]) == 2:
    if all_dao_info["df_proposals"]["scores_total"][i] != 0:
      percVP = max(all_dao_info["df_proposals"]["scores"][i])*100 / all_dao_info["df_proposals"]["scores_total"][i]
      myDictVP[all_dao_info["df_proposals"]["id"][i]] = percVP

#aqui hay que unir x e y en funcion del id de la propuesta
x = []
y = []

for i in myDictVP.items():
  for j in myDictVotos.items():
    if i[0] == j[0]:
      y.append(i[1])
      x.append(j[1])

plt.style.use('bmh')
plt.axes().set_aspect('equal')
plt.scatter(x, y, alpha=0.3, s=150)
plt.xlim(0, 100)
plt.ylim(0, 100)


plt.ylabel('Porcentaje de poder de voto', fontsize=20)
plt.xlabel('Porcentaje de votos', fontsize=20)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.savefig("agree_Pillar.pdf", format="pdf", bbox_inches="tight")
plt.show()



In [ ]:
#Relación entre el número de votos y el poder de voto que recibe una propuesta
plt.rcParams["figure.figsize"]=(12,7)
x = []
y = []
ids = []
i=1
for prop_id, nousar in IDsPropsOrdenados:

  num_votos = all_dao_info['df_proposals'][all_dao_info['df_proposals']['id']==prop_id]['votes']
  vp = all_dao_info['df_proposals'][all_dao_info['df_proposals']['id'] == prop_id]['scores_total']

  x.append(num_votos)
  y.append(vp/num_votos)
  ids.append("P"+str(i))
  i+=1

plt.style.use('bmh')
plt.scatter(x, y, alpha=0.5, s=150)
plt.ylabel('Poder de voto medio', fontsize=20)
plt.xlabel('Cantidad de votos', fontsize=20)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.savefig("prop_vvp_pillar.pdf", format="pdf", bbox_inches="tight")

# for i, txt in enumerate(ids):
#     plt.annotate(txt, (x[i], y[i]))

#no pinta las propuestas sin votos

# **Autoría**

In [ ]:
i = 0
j = 0
for num_votes, rep in Counter(Counter(list(all_dao_info["df_proposals"]["author"])).values()).items():
   rects = plt.bar(num_votes, rep, 0.4)
   height = rects[0].get_height()
  #  plt.text(rects[0].get_x() + rects[0].get_width()/2., height,
  #             f'{rep}',
  #             ha='center', va='bottom')
   i = i+1
   if(j<rep):
     j = rep

plt.style.use('bmh')
plt.xlabel("Cantidad de propuestas creadas")
plt.ylabel("Número de autores")
plt.xticks(range(1,4,1))
plt.yticks(range(0,7,1))
plt.show()